# Data Access Methods

**Marty Hidas**

**Integrated Marine Observing System (IMOS) / Australian Ocean Data Network (AODN)**

This tutorial demostrates several ways data can be accessed remotely and loaded into a Python environment, including

* OPeNDAP
* OGC Web Feature Service (WFS)
* direct access to files on cloud storage (AWS S3)
* cloud-optimised formats Zarr & Parquet
* New OGC APIs

In [ ]:
# Import all the tools we need...

# For working with data
import pandas as pd
import xarray as xr

# For data that may be larger than the memory available on your computer...
import dask
import dask.dataframe as dd

# For accessing OGC Web Feature Service
from owslib.wfs import WebFeatureService

# For accessing AWS S3 cloud storage
import s3fs

# Plotting tools
import holoviews as hv
import hvplot.pandas
import hvplot.xarray

# For plotting geographic data & maps
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts
from cartopy import crs

# Use Matplotlib backend for slides web preview of notebook
# Comment out these lines to get the default interactive plots using Bokeh
hvplot.extension('matplotlib', compatibility='bokeh')
gv.extension('matplotlib')
gv.output(size=200)

# The old school way

The old (and still common) way to access data is to first download it to your computer and read it from there. 
This is easy for small datasets, but not always ideal:
* What if the data is bigger than your hard disk?
* What if you only need a small fraction of a dataset?
* What if the dataset is routinely updated and you want to re-run your analysis on the latest data?
* What if you want to run your analysis on another computer or in the cloud?

These days it is often more convenient to have data managed in a central location and access it remotely.
There are many ways this can be done. In this tutorial we will look at a few of the common ones, and some of the newer ones.

# OPeNDAP

* [OPeNDAP](https://www.opendap.org/about) stands for "Open-source Project for a Network Data Access Protocol"
* Provides access to metadata and data subsets via the Web without downloading an entire dataset
* Many tools that can read NetCDF files can also talk to an OPeNDAP URL directly

In Python, we can simply open the URL with `xarray`, then proceed with our analysis using the resulgting `Dataset` object.

Here we use an example from the [AODN THREDDS server](https://thredds.aodn.org.au/thredds/catalog.html).

In [ ]:
# Set up local data path
import os
TUTORIAL_BASEPATH = os.path.realpath('..')
DATA_BASEPATH = os.path.join(os.path.dirname(TUTORIAL_BASEPATH), 'shared', 'IMOS')

In [ ]:
# Local copy of data file, in case server is overloaded...
# opendap_url = os.path.join(DATA_BASEPATH,
#                           "IMOS_ANMN-NSW_TZ_20091029_PH100_FV02_TEMP-gridded-timeseries_END-20230316_C-20230520.nc")
# ds_mooring = xr.open_dataset(opendap_url)
# ds_mooring

In [ ]:
opendap_url = ("https://thredds.aodn.org.au/thredds/dodsC/"
               "IMOS/ANMN/NSW/PH100/gridded_timeseries/"
               "IMOS_ANMN-NSW_TZ_20091029_PH100_FV02_TEMP-gridded-timeseries_END-20230316_C-20230520.nc")

ds_mooring = xr.open_dataset(opendap_url)
ds_mooring

In [ ]:
print(ds_mooring.title)

This dataset is derived from repeated deployments of moored temperature loggers, binned to hourly intervals and interpolated to a fixed set of target depths. See the file metadata, or the associated [metadata record](https://catalogue-imos.aodn.org.au/geonetwork/srv/eng/catalog.search#/metadata/279a50e3-21a5-4590-85a0-71f963efab82) for more info.

In [ ]:
# Hourly averages x 12 depths x 13+ yr = over a million points to plot!
# Let's just look at a year's worth to speed things up...
ds_mooring.sel(TIME="2022").hvplot.scatter(x="TIME", y="DEPTH", c="TEMP",
                                           cmap="coolwarm", alpha=0.2,
                                           flip_yaxis=True, hover=False)

In [ ]:
# ... or we can look at the full timeseries of temperature at a single depth
ds_mooring.TEMP.sel(DEPTH=30).hvplot()

# Web Feature Service (WFS)

* A [standard](http://www.opengeospatial.org/standards/wfs) of the [Open Geospatial Consortium](http://www.opengeospatial.org/) (OGC)
* Allows tabular geospatial data to be accessed via the Web.
* A _feature_ has a _geometry_ (e.g. a point/line/polygon) indicating a geographic location, and a set of properties (e.g. temperature) 
* WFS allows filtering based on geometry or properties.
* In Python WFS and other OGC Web Services (OWS) can be accessed using the [`owslib`](https://pypi.org/project/OWSLib/) library

For example, most of the tabular data from the Australian Integrated Marine Observing System (IMOS) is available via WFS.

In [ ]:
wfs = WebFeatureService(url="https://geoserver-123.aodn.org.au/geoserver/wfs",
                        version="1.1.0")
wfs.identification.title

In [ ]:
# Each dataset is served as a separate "feature type":
print(f"There are {len(wfs.contents)} fature types, e.g.")
list(wfs.contents)[:10]

For now we'll assume we already know which featuretype we want. In this example we'll look at a dataset containing condicutivity-temperature-depth (CTD) profiles obtained at the National Reference Stations around Australia ([here](https://catalogue-imos.aodn.org.au/geonetwork/srv/eng/catalog.search#/metadata/7b901002-b1dc-46c3-89f2-b4951cedca48)'s a detailed metadata record)

In [ ]:
typename = 'imos:anmn_ctd_profiles_data'
wfs.get_schema(typename)['properties']

We can read in a subset of the data by specifying a bounding box (in this case near Sydney, Australia).
We'll get the result in CSV format so it's easy to read into a Pandas DataFrame.

First we'll ask for just 10 features, for a quick look at the data.

In [ ]:
xmin, xmax = 151.2, 151.25   # Port Hacking, near Sydney, NSW
ymin, ymax = -34.2, -34.1

response = wfs.getfeature(typename=typename,
                          bbox=(xmin, ymin, xmax, ymax),
                          maxfeatures=10,
                          outputFormat='csv')
df = pd.read_csv(response)
response.close()

df

In [ ]:
# Load local copy of CSV file returned...
# local_csv = os.path.join(DATA_BASEPATH, 'wfs_response1.csv')
# df = pd.read_csv(local_csv)
# df

We can also filter the data based on the values in specified columns (properties) and ask for only a subset of the columns to be returned. The filters need to be provided in XML format, but the `owslib` library allows us to construct them in a more Pythonic way.

Here we select only the profiles associated with the Port Hacking 100m mooring site, and only the data points flagged as "good data" by automated quality-control procedures.

In [ ]:
from owslib.etree import etree
from owslib.fes import PropertyIsEqualTo, And

filter = And([PropertyIsEqualTo(propertyname="site_code", literal="PH100"),
              PropertyIsEqualTo(propertyname="PRES_REL_quality_control", literal="1"),
              PropertyIsEqualTo(propertyname="TEMP_quality_control", literal="1"),
              PropertyIsEqualTo(propertyname="PSAL_quality_control", literal="1"),
              PropertyIsEqualTo(propertyname="CPHL_quality_control", literal="1")
             ])
filterxml = etree.tostring(filter.toXML(), encoding="unicode")

response = wfs.getfeature(typename=typename, filter=filterxml, outputFormat="csv",
                          propertyname=["TIME", "DEPTH", "TEMP", "PSAL", "CPHL"]
                         )
df = pd.read_csv(response, parse_dates=["TIME"])
response.close()

# the server adds a feature ID column we don't really need
df.drop(columns='FID', inplace=True)

In [ ]:
# Load local copy of CSV file returned...
# local_csv = os.path.join(DATA_BASEPATH, 'wfs_response2.csv')
# df = pd.read_csv(local_csv).drop(columns='FID')
# df

In [ ]:
df

In [ ]:
# We can explore the temperature, salinity and chlorophyll profiles on by one
temp_plot = df.hvplot(x="TEMP", y="DEPTH", groupby="TIME", flip_yaxis=True, legend=False)
psal_plot = df.hvplot(x="PSAL", y="DEPTH", groupby="TIME", flip_yaxis=True, legend=False)
cphl_plot = df.hvplot(x="CPHL", y="DEPTH", groupby="TIME", flip_yaxis=True, legend=False)

(temp_plot + psal_plot + cphl_plot)

In [ ]:
# We can also extract the temperature measurements at a fixed depth
# and compare to the timeseries from the mooring 
comp_depth = 20  # metres

df_sub = df[df.DEPTH.round() == comp_depth]
ctd_plot = df_sub.hvplot.scatter(x="TIME", y="TEMP", c="red")

mooring_plot = ds_mooring.TEMP.sel(DEPTH=comp_depth).hvplot()

mooring_plot * ctd_plot

Further examples?
* Plot timeseries of near-surface values
* Plot profile by month of year?
* Complute MLD (or read from `nrs_derived_indices_data`) and plot timeseries
* Calculate average profile per month of year?
* Plot timeseries of various phytoplankton species abundances?

**TODO** Add abstract & metadata link to the example WFS layer

# Direct access to files on cloud storage

Data files made available to the public on cloud storage such as [Amazon S3](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html) can be accessed over the web as if they were stored locally. You just need to find the exact URL for each file.

In Python, we can access S3 storage in a very similar way to a local filesystem using the `s3fs` library.

For example, all the public data files hosted by the Australian Ocean Data Network are stored in an [S3 bucket](https://www.techtarget.com/searchaws/definition/AWS-bucket) called `imos-data`. You can browse the contents of the bucket and download individual files [here](https://imos-data.aodn.org.au). 

Below we'll look at a [high-resolution regional SST product](https://catalogue-imos.aodn.org.au/geonetwork/srv/eng/catalog.search#/metadata/a4170ca8-0942-4d13-bdb8-ad4718ce14bb) from IMOS (based on satellite and in-situ observations). This product is a collection of daily gridded NetCDF files covering the Australian region.

In [ ]:
s3 = s3fs.S3FileSystem(anon=True)

# List the most recent files available
sst_files = s3.ls("imos-data/IMOS/SRS/SST/ghrsst/L4/RAMSSA/2023")
sst_files[-20:]

In [ ]:
# Open the latest file and look at its contents
ds = xr.open_dataset(s3.open(sst_files[-1]))
ds

In [ ]:
# Plot a subset of the dataset around Australia
sst_var = 'analysed_sst'
gds = gv.Dataset(ds.sel(lat=slice(-50, 0), lon=slice(105, 175)),
                 kdims=['lon', 'lat'],
                 vdims=[sst_var]
                )
sst_plot = (gds.to(gv.Image)
               .opts(cmap='coolwarm', colorbar=True, aspect=1.4, title=ds.title))
sst_plot * gf.land

It's worth understanding a little about how this works. 

The above example only makes use of the metadata from the file, one of the 4 data variables, and the `lon` and `lat` coordinates. On a local filesystem, it would be easy to read only these specific parts of the file from disk. 

However, on cloud storage services like S3 (also called "object storage") the basic read/write functions operate on the entire file (object), so at least in the backend, the entire file is read**. If you only need a small subset of a large file, this can be a very inefficient way to get it.

** _Note: it is possible to request only a subset of an S3 object to be read, but this is more advanced usage than what we're doing here._

For example, if we wanted to plot a timeseries of the above satellite SST product at a given point, we would only need a single value out of each file (corresponding to one point in the timeseries), but the entire file would need to be read each time.

For a quick demo we'll try this with last month's files. `xarray` has a handy `open_mfdataset` function that can create a single `Dataset` object out of a series of files (with similar structure).

In [ ]:
%%time
s3_objs = [s3.open(f)
           for f in s3.glob("imos-data/IMOS/SRS/SST/ghrsst/L4/RAMSSA/2023/202307*")
          ]
mds = xr.open_mfdataset(s3_objs, engine="h5netcdf")
mds

The variables in the dataset are not loaded into memory (they're still `dask.array`s). However, in the background, each complete file had to be downloaded from S3 before the metadata needed by `open_mfdataset` could be read. 

In [ ]:
mds.analysed_sst

Let's compare this to reading the same files from a local filesystem...

In [ ]:
%%time
from glob import glob
local_files = glob(os.path.join(DATA_BASEPATH, "RAMSSA", "*"))

mds = xr.open_mfdataset(local_files, engine="h5netcdf")
mds

Whichever way we loaded the dataset, we can plot it the same way as any other `xarray.Dataset`.

In [ ]:
%%time
mds[sst_var].sel(lat=-42, lon=150, method="nearest").hvplot()

## Zarr - a cloud-optimised data format

Zarr is a relatively new data format specifically developed for efficient access to multi-dimensional data in the cloud. Each dataset is broken up into many smaller files containing "chunks" of the data, organised in a standard hierarchy. The metadata are stored in separate files. When reading such a dataset, only the required information is read for each operation.

In [ ]:
# A Zarr "store" can easily be opened as an xarray.Dataset

# In this case the Zarr store is in an S3 bucket
store = s3fs.S3Map(root='imos-data-pixeldrill/zarrs/2021/', s3=s3, check=False)

zds = xr.open_zarr(store)
zds

In [ ]:
# We can see the chunked structure of the data by looking at one of the variables
zds.sea_surface_temperature

In [ ]:
%%time

# We can plot this dataset in exactly the same way as the NetCDF-based one
sst_var = 'sea_surface_temperature'
gds = gv.Dataset(zds[sst_var].sel(time='2021-01-02', lat=slice(0, -50), lon=slice(105, 175)),
                 kdims=['lon', 'lat'],
                 vdims=[sst_var]
                )
sst_plot = (gds.to(gv.Image, ['lon', 'lat'])
               .opts(cmap='coolwarm', colorbar=True, aspect=1.4, title=zds.title))
sst_plot * gf.land

A more detailed example of working with similar data in Zarr format can be found here: https://github.com/aodn/rimrep-examples/blob/main/Python_based_scripts/zarr.ipynb

## Parquet

* Parquet is a cloud-optimised format designed for tabular data.
* Each column of the table is stored in a separate file/object.
* These can be further partitioned into _row groups_.

For a quick demo, we'll borrow an example from [this more detailed notebook](https://github.com/aodn/rimrep-examples/blob/main/Python_based_scripts/Extracting_Water_Temperature_at_Site.ipynb), looking at temperature logger data from the Australian Institute of Marine Science. The dataset contains 150 million temperature measurements from numerous sites around Australia ([metadata for this dataset](https://apps.aims.gov.au/metadata/view/4a12a8c0-c573-11dc-b99b-00008a07204e)).

In [ ]:
# Here's the path to the dataset on AWS S3
parquet_path = "s3://rimrep-data-public/091-aims-sst/test-50-64-spatialpart/"

# Let's see if there are any temperature loggers near us (in Dunsborough, Western Australia)
filters = [('lon', '>', 114.5),
          ('lon', '<', 115.5),
          ('lat', '>', -34.),
          ('lat', '<', -33.)]

df = dd.read_parquet(parquet_path,
                     filters=filters,
                     # only read the site names and QC'd temperature values
                     columns = ['site', 'qc_val'],
                     index='time',
                     storage_options = {"anon": True}
                    )
df.head()

In [ ]:
len(df)

In [ ]:
# We still have 1.6 million values.
# Let's see how many sites we have...
df.site.unique().compute()  # need compute() for dask to give us an answer now

In [ ]:
# Plot a timeseries for the Geographe Bay site
df_local = df[df.site == "Geographe Bay"].compute()
df_local.hvplot()

#### Alternative dataset

Another Parquet example from OBIS is shown in [this notebook](https://github.com/MathewBiddle/globe/blob/main/organismQuantity_check.ipynb)

# Other methods

* ERDDAP
* New OGC API

# TODO

- [ ] Add metadata links for datasets used
- [ ] Find alternative data sources in other regions (at least US)
- [ ] Provide sample data to store on JupyterHub for local access
- [ ] Provide clear instructions for participants which data to access!
- [ ] Acknowledge previous tutorial and other sources...
- [ ] Data acknowledgements / citations?
- [ ] Spell check